In [8]:
!pip install langchain openai langchain_openai langchain_core langchain_chains

ERROR: Could not find a version that satisfies the requirement langchain_chains (from versions: none)
ERROR: No matching distribution found for langchain_chains


In [9]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "sk-proj-67OykLCevETcugz42l3JT3BlbkFJKYocTIEv1IAjDGO8Eqcl"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [10]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi! how are you doing?"),
]

model.invoke(messages)

AIMessage(content='Ciao! Come stai?', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 25, 'total_tokens': 32}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-73bb2f75-e758-4438-971f-f798062d8543-0', usage_metadata={'input_tokens': 25, 'output_tokens': 7, 'total_tokens': 32})

**Parsing the output**

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao! Come stai?'

**Prompt Template**

In [14]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi, how are you?"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi, how are you?')])

In [15]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi, how are you?')]

**Chaining together components with LCEL**

In [18]:
message = 'hi, how are you?'

chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": message})

'ciao, come stai?'

### **Building a ChatBot**

In [19]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo")

In [20]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 12, 'total_tokens': 22}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c50b475b-cbd6-4224-bc94-81f2267110af-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [21]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, I do not have access to that information. Can you please provide me with your name?", response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 12, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-7344106f-acea-40ed-bf4f-7ac31a8cfd5a-0', usage_metadata={'input_tokens': 12, 'output_tokens': 22, 'total_tokens': 34})

In [22]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 35, 'total_tokens': 40}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e18d4036-1c77-430d-99f6-3b31f53e5f73-0', usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40})

**Message History**
<BR>
We can use a Message History class to wrap our model and make it stateful

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {"configurable": {"session_id": "abc2"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Now the chatbot  **remembers** things about us. <br>
If we change the config to reference a different session_id, we can see that it starts the conversation fresh.

In [ ]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content